# Demonstration of peeling

In [ ]:
% matplotlib inline

import os
import sys
sys.path.append(os.path.join('..','..'))

results_dir = './results'
os.makedirs(results_dir, exist_ok=True)

from matplotlib import pylab

pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs.utils import pixel_to_skycoord

from matplotlib.pyplot import cm

from arl.data.data_models import Skycomponent
from arl.data.polarisation import PolarisationFrame
from arl.visibility.base import create_blockvisibility
from arl.skycomponent.operations import create_skycomponent, apply_beam_to_skycomponent
from arl.image.operations import show_image, export_image_to_fits, qa_image
from arl.imaging.base import create_image_from_visibility
from arl.fourier_transforms.fft_support import extract_mid
from arl.visibility.coalesce import coalesce_visibility
from arl.image.iterators import image_raster_iter
from arl.imaging.base import predict_skycomponent_visibility
from arl.visibility.iterators import vis_timeslice_iter
from arl.util.testing_support import create_named_configuration, create_low_test_beam, \
    create_low_test_skycomponents_from_gleam, simulate_gaintable
from arl.imaging.imaging_context import invert_function
from arl.calibration.operations import create_gaintable_from_blockvisibility, apply_gaintable
from arl.calibration.peeling import peel_skycomponent_blockvisibility

import logging

log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))


Make the visibility dataset

In [ ]:
df = 1e6
frequency = numpy.array([1e8 - df, 1e8, 1e8 + df])
channel_bandwidth = numpy.array([df, df, df])

# Define the component and give it some spectral behaviour
f = numpy.array([100.0, 20.0, -10.0, 1.0])
flux = numpy.array([f, 0.8 * f, 0.6 * f])
phasecentre = SkyCoord(ra=+15.0 * u.deg, dec=-35.0 * u.deg, frame='icrs', equinox='J2000')


config = create_named_configuration('LOWBD2-CORE')

times = numpy.linspace(-3.0, 3.0, 7) * numpy.pi / 12.0
vis = create_blockvisibility(config, times, frequency,
                                  channel_bandwidth=channel_bandwidth,
                                  phasecentre=phasecentre, weight=1.0,
                                  polarisation_frame=PolarisationFrame('linear'))
vis.data['vis'][...]=0.0

Create a peelable source, predict the visibility, simulate a gain table and then apply it to the visibilities. 

In [ ]:
peeldirection = phasecentre = SkyCoord(ra=+20.0 * u.deg, dec=-45.0 * u.deg, frame='icrs', equinox='J2000')


peel = Skycomponent(direction=peeldirection, frequency=frequency, flux=flux,
                    polarisation_frame=PolarisationFrame("stokesIQUV"))

vis = predict_skycomponent_visibility(vis, peel)
gt = create_gaintable_from_blockvisibility(vis)
gt = simulate_gaintable(gt, phase_error=1.0, amplitude_error=0.01)
vis = apply_gaintable(vis, gt)

Make the set of components that will define the main field. Then just add their visibilities to the data set.

In [ ]:
sc = create_low_test_skycomponents_from_gleam(flux_limit=1.0,
                                              polarisation_frame=PolarisationFrame("stokesIQUV"),
                                              frequency=frequency, kind='cubic',
                                              phasecentre=phasecentre,
                                              radius=0.5)
model = create_image_from_visibility(vis, cellsize=0.001, npixel=512, frequency=frequency,
                                     polarisation_frame=PolarisationFrame('stokesIQUV'))

bm = create_low_test_beam(model=model)
sc = apply_beam_to_skycomponent(sc, bm, flux_limit=0.1)
vis = predict_skycomponent_visibility(vis, sc)

Make the dirty image

In [ ]:
dirty, sumwt = invert_function(vis, model, context='timeslice', timeslice='auto')
show_image(dirty)
export_image_to_fits(dirty, "%s/peeling_dirty.fits" % (results_dir))

Finally we can peel out the source.

In [ ]:
vis, peel_gts = peel_skycomponent_blockvisibility(vis, peel)

print(peel_gts[0].residual[:,0,...,0,0])

In [ ]:
dirty, sumwt=invert_function(vis, model, context='timeslice', timeslice='auto')
show_image(dirty)
export_image_to_fits(dirty, "%s/peeling_dirty_peeled.fits" % (results_dir))

In [ ]:
print(qa_image(dirty))